In [ ]:
from wc_simd.llm import get_llm
import os

# Set LANGSMITH_PROJECT environment variable
os.environ['LANGSMITH_PROJECT'] = "agent_playground"

llm = get_llm()

# Supervisor-style Agent

In [8]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool

# Create specialized agents

@tool
def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

@tool
def web_search(query: str) -> str:
    """Search the web for information."""
    return (
        "Here are the headcounts for each of the FAANG companies in 2024:\n"
        "1. **Facebook (Meta)**: 67,317 employees.\n"
        "2. **Apple**: 164,000 employees.\n"
        "3. **Amazon**: 1,551,000 employees.\n"
        "4. **Netflix**: 14,000 employees.\n"
        "5. **Google (Alphabet)**: 181,269 employees."
    )


math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

web_search_agent = create_react_agent(
    model=llm,
    tools=[web_search],
    name="web_search_expert",
    prompt="You are a web search expert. Do not do anything else other than web search."
)

# Create supervisor workflow
workflow = create_supervisor(
    [web_search_agent, math_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a web search expert and a math expert. "
        "For current events, use web_search_expert. "
        "For math problems, use math_agent."
    )
)

# Compile and run
app = workflow.compile()

In [9]:
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what's the combined headcount of the FAANG companies in 2024?"
        }
    ]
})
result

{'messages': [HumanMessage(content="what's the combined headcount of the FAANG companies in 2024?", additional_kwargs={}, response_metadata={}, id='02bea936-c5c6-435a-9a9a-0645fdaf0097'),
  AIMessage(content="I'll help you find the combined headcount of the FAANG companies (Facebook/Meta, Apple, Amazon, Netflix, Google/Alphabet) in 2024. This requires current information about employee counts at these major tech companies, so I'll transfer your question to our web search expert.", additional_kwargs={'usage': {'prompt_tokens': 473, 'completion_tokens': 104, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 577}, 'stop_reason': 'tool_use', 'thinking': {}, 'model_id': 'eu.anthropic.claude-3-7-sonnet-20250219-v1:0', 'model_name': 'eu.anthropic.claude-3-7-sonnet-20250219-v1:0'}, response_metadata={'usage': {'prompt_tokens': 473, 'completion_tokens': 104, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 577}, 'stop_reason': 'tool_use', '